In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase,Predictor,PretrainedPredictor
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy

from scipy import stats

In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Tamoxifen'
na = 1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSC2_expression.csv',index_col=0)
label_r=pd.read_csv('data/GDSC2_label_9drugs.csv',index_col=0)

In [5]:
label_r=label_r.fillna(na)

In [6]:
hvg,adata = ut.highly_variable_genes(data_r)

In [7]:
selected_idx = label_r.loc[:,select_drug]!=na

In [8]:
data_r.columns = adata.var_names

In [9]:
hvg.sum()

5116

# Your is gene-cell, mine is cell-gene

In [10]:
#data = data_r.loc[selected_idx,:]
data = data_r.loc[selected_idx,hvg]

In [11]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()
lbscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = lbscaler.fit_transform(label.values.reshape(-1,1))
#label = label.values.reshape(-1,1)

In [12]:
print(np.std(data))
print(np.mean(data))

0.25212143569762396
0.24222409715211315


In [13]:
data.mean(axis=0)

array([0.50048141, 0.07888812, 0.20731304, ..., 0.11936042, 0.05345683,
       0.403317  ])

In [14]:
print(data.max())
print(data.min())

1.0000000000000004
0.0


In [15]:
data.shape

(753, 5116)

In [16]:
label_r.shape

(804, 9)

# Split test train

In [17]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [18]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(753, 5116)
(753, 1)
(481, 5116) (481, 1)
(151, 5116) (151, 1)


In [19]:
print(X_train.max())
print(X_train.min())

1.0000000000000004
0.0


# AE MODEL

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [21]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)

Y_trainTensor = torch.FloatTensor(Y_train).to(device)
Y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [22]:
dataloader = X_trainDataLoader

In [23]:
X_trainDataLoader.dataset.tensors[0].shape[0]

481

# The model

In [24]:
model = PretrainedPredictor(input_dim=5116,latent_dim=512,hidden_dims=[2048,1024], 
                            hidden_dims_predictor=[256,128],
                            pretrained_weights='saved/models/GDSCnew_ae.pkl',freezed=False)

In [25]:
print(model)

PretrainedPredictor(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=5116, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2048, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=1024, out_features=1, bias=True)
  (decoder_input): Linear(in_features=1, out_features=1024, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (final_layer): Sequential(
    (0): Linear(in_features=2048, out_features=5116, bias=True)
    (1):

In [26]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [27]:
# Load data
# data type conversion

# y_trainTensor = torch.FloatTensor(Y_train).to(device)
# y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=200, shuffle=True)
validDataLoader_p = DataLoader(dataset=validreducedDataset, batch_size=200, shuffle=True)

In [28]:
dataloaders_train = {'train':trainDataLoader_p,'val':validDataLoader_p}

In [ ]:
model,report = ut.train_predictor_model(model,dataloaders_train,
                                        optimizer,loss_function,epochs,exp_lr_scheduler,save_path="saved/models/pre_pre_model.pkl")

Epoch 0/499
----------
train Loss: 0.00140895. Learning rate = 0.01
val Loss: 0.00143214. Learning rate = 0.01
Epoch 1/499
----------
train Loss: 0.00161581. Learning rate = 0.01
val Loss: 0.00143214. Learning rate = 0.01
Epoch 2/499
----------
train Loss: 0.00121800. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 3/499
----------
train Loss: 0.00117418. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 4/499
----------
train Loss: 0.00112455. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 5/499
----------
train Loss: 0.00114035. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 6/499
----------
train Loss: 0.00115762. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 7/499
----------
train Loss: 0.00107312. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
Epoch 8/499
----------
train Loss: 0.00110942. Learning rate = 0.01
val Loss: 0.00147701. Learning rate = 0.01
E

Epoch 72/499
----------
train Loss: 0.00109084. Learning rate = 1e-05
val Loss: 0.00147701. Learning rate = 1e-05
Epoch 73/499
----------
train Loss: 0.00112819. Learning rate = 1e-05
val Loss: 0.00147701. Learning rate = 1e-05
Epoch 74/499
----------
train Loss: 0.00111972. Learning rate = 1.0000000000000002e-06
val Loss: 0.00147701. Learning rate = 1.0000000000000002e-06
Epoch 75/499
----------
train Loss: 0.00110815. Learning rate = 1.0000000000000002e-06
val Loss: 0.00147701. Learning rate = 1.0000000000000002e-06
Epoch 76/499
----------
train Loss: 0.00108824. Learning rate = 1.0000000000000002e-06
val Loss: 0.00147701. Learning rate = 1.0000000000000002e-06
Epoch 77/499
----------
train Loss: 0.00114563. Learning rate = 1.0000000000000002e-06
val Loss: 0.00147701. Learning rate = 1.0000000000000002e-06
Epoch 78/499
----------
train Loss: 0.00106501. Learning rate = 1.0000000000000002e-06
val Loss: 0.00147701. Learning rate = 1.0000000000000002e-06
Epoch 79/499
----------
train Lo

Epoch 128/499
----------
train Loss: 0.00112382. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 129/499
----------
train Loss: 0.00111260. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 130/499
----------
train Loss: 0.00107878. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 131/499
----------
train Loss: 0.00113032. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 132/499
----------
train Loss: 0.00112158. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 133/499
----------
train Loss: 0.00113036. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 134/499
----------
train Loss: 0.00109392. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

Epoch 183/499
----------
train Loss: 0.00108665. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 184/499
----------
train Loss: 0.00111291. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 185/499
----------
train Loss: 0.00111476. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 186/499
----------
train Loss: 0.00110711. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 187/499
----------
train Loss: 0.00111737. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 188/499
----------
train Loss: 0.00110446. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 189/499
----------
train Loss: 0.00108497. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

Epoch 238/499
----------
train Loss: 0.00116311. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 239/499
----------
train Loss: 0.00114852. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 240/499
----------
train Loss: 0.00119874. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 241/499
----------
train Loss: 0.00117429. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 242/499
----------
train Loss: 0.00107574. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 243/499
----------
train Loss: 0.00115942. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 244/499
----------
train Loss: 0.00117065. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

Epoch 293/499
----------
train Loss: 0.00109542. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 294/499
----------
train Loss: 0.00107561. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 295/499
----------
train Loss: 0.00116081. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 296/499
----------
train Loss: 0.00112081. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 297/499
----------
train Loss: 0.00109453. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 298/499
----------
train Loss: 0.00114795. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 299/499
----------
train Loss: 0.00113710. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

Epoch 348/499
----------
train Loss: 0.00110112. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 349/499
----------
train Loss: 0.00107595. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 350/499
----------
train Loss: 0.00114959. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 351/499
----------
train Loss: 0.00111313. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 352/499
----------
train Loss: 0.00114814. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 353/499
----------
train Loss: 0.00110495. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 354/499
----------
train Loss: 0.00111255. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

Epoch 403/499
----------
train Loss: 0.00107854. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 404/499
----------
train Loss: 0.00106129. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 405/499
----------
train Loss: 0.00113415. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 406/499
----------
train Loss: 0.00114206. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 407/499
----------
train Loss: 0.00107288. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 408/499
----------
train Loss: 0.00111479. Learning rate = 1.0000000000000004e-08
val Loss: 0.00147701. Learning rate = 1.0000000000000004e-08
Epoch 409/499
----------
train Loss: 0.00116768. Learning rate = 1.0000000000000004e-08
val Loss: 0.001477

In [ ]:
model(X_testTensor)

In [ ]:
Y_test

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
dl_result = model(X_testTensor).detach().cpu().numpy()

In [ ]:
r2_score(dl_result,Y_test)

In [ ]:
pearsonr(dl_result.flatten(),Y_test.flatten())

In [ ]:
mean_squared_error(dl_result,Y_test)